# Support Vector Machines

SVM is another type of classifier which seprates the data-point in the feature space using hyperplanes. When we look at the graph showing SVM there is a weight vector which lets us change the orientation of the hyperplane and the bias which lets us change the distance of hyperplane from the origin. **So our task is to find the appropriate bias and the weight vector so that the datapoints get seprated in two classes!**

Now in 1957 Frank Rosenblatt modeled the equation wx + b = 0 as a perceptron and his hypothesis was sadly rejected(*because of the XOR problem*). But then in recent years a concept of adding minimum dimention to seprate the datapoints by doing a trick and using a hyperplane revived it which is called deep learning now. 

## Something is weird about SVMs

First, the motive of support vectors machines is to find maximum margin hyperplane, and the data points which touch this hyperplane(*which'll be the border points*) are called the support vectors. **So SVM only depend on the border cases**
Which is weird! 

### Really, really refer to the slides!